目的

Abe_speechで取集した生データから
「○安倍晋三君　自由民主党の安倍晋三であります。」
の部分を消去する

その後処理後のファルダに入れる

やりたい事
各文書を形態素解析して
品詞ごとに出力回数降順に並べる。
その結果を何らかの方法でまとめる

例えば、Excelデータで
月毎　動詞　名詞　形容詞　末尾表現　
１
２
３
４
５

みたいな

In [42]:
def new_mecab(fname):
    Matubi=[]
    with open(fname,errors='ignore') as data_file:
                ftitle, fext = os.path.splitext(fname)
                fname_out=ftitle+'Prep.txt'
                #空白文字を削除する
                text_data=data_file.read()
                text_data=re.sub('.+　','',text_data)
                #改行で区切る
                text_data=re.sub('\n','',text_data)
                #句点で区切り、最後の要素を抜き取る
                text_data=text_data.split('。')
                with open(fname_out, mode='w',errors='ignore') as out_file:
                    for i,text in enumerate(text_data):
                        #読点以前を削除する
    #                     text=text.split('、')
                        #なぜか先頭に空白文字が入るので取り除く↓。
                        text=text.lstrip()
                        #読点以降のみが集まる↓
                        text+='。'
                        out_file.write(text)
                    

# new_mecab (r'C:\Users\icech\Desktop\share\Lab\2018_09_05\Docments\Abe_speech\2012-10-01.txt')    

for file in glob.glob(r'C:\Users\icech\Desktop\share\Lab\2018_09_05\Docments\Abe_speech/*'):
    new_mecab(file)

In [3]:
from 過去のコード.Mecab_func import *
import glob
import re
import sys
import os
import pandas as pd

In [56]:
#前処理したTxtファイルからmecabファイルを作成するコード
# for file in glob.glob(r'C:\Users\icech\Desktop\share\Lab\2018_09_05\Docments\Abe_speech\Prep\*'):
    ftitle, fext = os.path.splitext(file)
    fname_out=ftitle+'.mecab'
    to_mecab(file,fname_out)

In [29]:
def make_stopword():
    f = open("../docments/stopword.txt","r")
    list = []
    for x in f:
        list.append(x.rstrip("\n"))
        #以下のようにしてしまうと、改行コードがlistに入ってしまうため注意
        #list.append(x)
    f.close()
    return(list)

print(make_stopword())

['あそこ', 'あたり', 'あちら', 'あっち', 'あと', 'あな', 'あなた', 'あれ', 'いくつ', 'いつ', 'いま', 'いや', 'いろいろ', 'うち', 'おおまか', 'おまえ', 'おれ', 'がい', 'かく', 'かたち', 'かやの', 'から', 'がら', 'きた', 'くせ', 'ここ', 'こっち', 'こと', 'ごと', 'こちら', 'ごっちゃ', 'これ', 'これら', 'ごろ', 'さまざま', 'さらい', 'さん', 'しかた', 'しよう', 'すか', 'ずつ', 'すね', 'すべて', 'ぜんぶ', 'そう', 'そこ', 'そちら', 'そっち', 'そで', 'それ', 'それぞれ', 'それなり', 'たくさん', 'たち', 'たび', 'ため', 'だめ', 'ちゃ', 'ちゃん', 'てん', 'とおり', 'とき', 'どこ', 'どこか', 'ところ', 'どちら', 'どっか', 'どっち', 'どれ', 'なか', 'なかば', 'なに', 'など', 'なん', 'はじめ', 'はず', 'はるか', 'ひと', 'ひとつ', 'ふく', 'ぶり', 'べつ', 'へん', 'ぺん', 'ほう', 'ほか', 'まさ', 'まし', 'まとも', 'まま', 'みたい', 'みつ', 'みなさん', 'みんな', 'もと', 'もの', 'もん', 'やつ', 'よう', 'よそ', 'わけ', 'わたし', '', 'ハイ', '', '', '上', '中', '下', '字', '', '', '年', '月', '日', '時', '分', '秒', '週', '火', '水', '木', '金', '土', '国', '都', '道', '府', '県', '市', '区', '町', '村', '', '', '各', '第', '方', '何', '的', '度', '文', '者', '性', '体', '人', '他', '今', '部', '課', '係', '外', '類', '達', '気', '室', '口', '誰', '用', '界', '会', '首', '男', '女', '別', '話', '私', '屋', '店', '家', '場'

In [30]:
def new_count_morpheme(word_class,mecab_file):
    copus=[]
    stopword=make_stopword()
    #語数カウンター
    word_counter = Counter()
    for line in make_lines(mecab_file):
        for morpheme in line:
            if len(morpheme['base'])==1 or morpheme['base'] in stopword:
                continue
            if morpheme['pos'] == word_class:
                word_counter.update([morpheme['base']])
                #表層型をリスト型にする
                copus.append(morpheme['base'])
                copus.append(' ')
    copus=','.join(copus)
    return(copus,word_counter)

In [31]:
#mecabファイルを読み込んで語数を数えるコード
df1=pd.DataFrame()
for file in glob.glob(r'C:\Users\icech\Desktop\share\Lab\2018_09_05\Docments\Abe_speech\Mecab\*'):
    month=re.sub( '.+Mecab.' ,'', file)
    month=re.sub('-01Prep.+', '' ,month)
    print(month)
    
    adjective_word,adjective_cnt=new_count_morpheme('形容詞',file)
    verb_word,verb_cnt=new_count_morpheme('動詞',file)
    noun_word,noun_cnt=new_count_morpheme('名詞',file)

    adjective_word ,adjective_cnt =zip(*adjective_cnt.most_common())
    try:
        verb_word ,verb_cnt =zip(*verb_cnt.most_common())
    except:
        pass   
    noun_word ,noun_cnt =zip(*noun_cnt.most_common())
    
    adjective_data={'{}adj_cnt'.format(month):list(adjective_cnt),
                    'adj_word':list(adjective_word),
        }
    adjective_df=pd.DataFrame(adjective_data)

    verb_data={'verb_cnt':list(verb_cnt),
               'verb_word':list(verb_word)
              }        
    
    verb_df=pd.DataFrame(verb_data)

    noun_data={'noun_cnt':list(noun_cnt),
               'noun_word':list(noun_word),
               'space':list(np.zeros(len(noun_cnt)))
        }
    noun_df=pd.DataFrame(noun_data)
    
    df2=pd.concat([df1,adjective_df,verb_df,noun_df],axis=1)
    print(df2)
    df1=df2.copy()



2012-10


c:\users\icech\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: generator 'make_lines' raised StopIteration
  


     2012-10adj_cnt adj_word  verb_cnt verb_word  noun_cnt  noun_word  space
0              45.0       強い      70.0       られる       155         総理    0.0
1              20.0      新しい      65.0       できる       105         国民    0.0
2              15.0      大きい      55.0       いたす        80       野田総理    0.0
3              10.0       重い      45.0       つくる        70       みずから    0.0
4              10.0      力強い      35.0       続ける        60         責任    0.0
5              10.0      美しい      35.0       果たす        60         政権    0.0
6               5.0      著しい      35.0       述べる        50         約束    0.0
7               5.0       軽い      30.0        示す        50       日米同盟    0.0
8               5.0    恥ずかしい      30.0       迎える        45         外交    0.0
9               5.0       若い      30.0        図る        40         結果    0.0
10              5.0    揺るぎない      25.0        とる        35        皆さん    0.0
11              5.0      つらい      25.0      取り戻す        35         解散    0.0

      2012-10adj_cnt  adj_word verb_cnt  verb_word noun_cnt  noun_word space  \
0               45.0        強い     70.0        られる    155.0         総理   0.0   
1               20.0       新しい     65.0        できる    105.0         国民   0.0   
2               15.0       大きい     55.0        いたす     80.0       野田総理   0.0   
3               10.0        重い     45.0        つくる     70.0       みずから   0.0   
4               10.0       力強い     35.0        続ける     60.0         責任   0.0   
5               10.0       美しい     35.0        果たす     60.0         政権   0.0   
6                5.0       著しい     35.0        述べる     50.0         約束   0.0   
7                5.0        軽い     30.0         示す     50.0       日米同盟   0.0   
8                5.0     恥ずかしい     30.0        迎える     45.0         外交   0.0   
9                5.0        若い     30.0         図る     40.0         結果   0.0   
10               5.0     揺るぎない     25.0         とる     35.0        皆さん   0.0   
11               5.0       つらい     25.0 

      2012-10adj_cnt  adj_word verb_cnt  verb_word noun_cnt  noun_word space  \
0               45.0        強い     70.0        られる    155.0         総理   0.0   
1               20.0       新しい     65.0        できる    105.0         国民   0.0   
2               15.0       大きい     55.0        いたす     80.0       野田総理   0.0   
3               10.0        重い     45.0        つくる     70.0       みずから   0.0   
4               10.0       力強い     35.0        続ける     60.0         責任   0.0   
5               10.0       美しい     35.0        果たす     60.0         政権   0.0   
6                5.0       著しい     35.0        述べる     50.0         約束   0.0   
7                5.0        軽い     30.0         示す     50.0       日米同盟   0.0   
8                5.0     恥ずかしい     30.0        迎える     45.0         外交   0.0   
9                5.0        若い     30.0         図る     40.0         結果   0.0   
10               5.0     揺るぎない     25.0         とる     35.0        皆さん   0.0   
11               5.0       つらい     25.0 

...

      2012-10adj_cnt  adj_word verb_cnt  verb_word noun_cnt  noun_word space  \
0               45.0        強い     70.0        られる    155.0         総理   0.0   
1               20.0       新しい     65.0        できる    105.0         国民   0.0   
2               15.0       大きい     55.0        いたす     80.0       野田総理   0.0   
3               10.0        重い     45.0        つくる     70.0       みずから   0.0   
4               10.0       力強い     35.0        続ける     60.0         責任   0.0   
5               10.0       美しい     35.0        果たす     60.0         政権   0.0   
6                5.0       著しい     35.0        述べる     50.0         約束   0.0   
7                5.0        軽い     30.0         示す     50.0       日米同盟   0.0   
8                5.0     恥ずかしい     30.0        迎える     45.0         外交   0.0   
9                5.0        若い     30.0         図る     40.0         結果   0.0   
10               5.0     揺るぎない     25.0         とる     35.0        皆さん   0.0   
11               5.0       つらい     25.0 

      2012-10adj_cnt  adj_word verb_cnt  verb_word noun_cnt  noun_word space  \
0               45.0        強い     70.0        られる    155.0         総理   0.0   
1               20.0       新しい     65.0        できる    105.0         国民   0.0   
2               15.0       大きい     55.0        いたす     80.0       野田総理   0.0   
3               10.0        重い     45.0        つくる     70.0       みずから   0.0   
4               10.0       力強い     35.0        続ける     60.0         責任   0.0   
5               10.0       美しい     35.0        果たす     60.0         政権   0.0   
6                5.0       著しい     35.0        述べる     50.0         約束   0.0   
7                5.0        軽い     30.0         示す     50.0       日米同盟   0.0   
8                5.0     恥ずかしい     30.0        迎える     45.0         外交   0.0   
9                5.0        若い     30.0         図る     40.0         結果   0.0   
10               5.0     揺るぎない     25.0         とる     35.0        皆さん   0.0   
11               5.0       つらい     25.0 

      2012-10adj_cnt  adj_word verb_cnt  verb_word noun_cnt  noun_word space  \
0               45.0        強い     70.0        られる    155.0         総理   0.0   
1               20.0       新しい     65.0        できる    105.0         国民   0.0   
2               15.0       大きい     55.0        いたす     80.0       野田総理   0.0   
3               10.0        重い     45.0        つくる     70.0       みずから   0.0   
4               10.0       力強い     35.0        続ける     60.0         責任   0.0   
5               10.0       美しい     35.0        果たす     60.0         政権   0.0   
6                5.0       著しい     35.0        述べる     50.0         約束   0.0   
7                5.0        軽い     30.0         示す     50.0       日米同盟   0.0   
8                5.0     恥ずかしい     30.0        迎える     45.0         外交   0.0   
9                5.0        若い     30.0         図る     40.0         結果   0.0   
10               5.0     揺るぎない     25.0         とる     35.0        皆さん   0.0   
11               5.0       つらい     25.0 

      2012-10adj_cnt  adj_word verb_cnt  verb_word noun_cnt  noun_word space  \
0               45.0        強い     70.0        られる    155.0         総理   0.0   
1               20.0       新しい     65.0        できる    105.0         国民   0.0   
2               15.0       大きい     55.0        いたす     80.0       野田総理   0.0   
3               10.0        重い     45.0        つくる     70.0       みずから   0.0   
4               10.0       力強い     35.0        続ける     60.0         責任   0.0   
5               10.0       美しい     35.0        果たす     60.0         政権   0.0   
6                5.0       著しい     35.0        述べる     50.0         約束   0.0   
7                5.0        軽い     30.0         示す     50.0       日米同盟   0.0   
8                5.0     恥ずかしい     30.0        迎える     45.0         外交   0.0   
9                5.0        若い     30.0         図る     40.0         結果   0.0   
10               5.0     揺るぎない     25.0         とる     35.0        皆さん   0.0   
11               5.0       つらい     25.0 

      2012-10adj_cnt  adj_word verb_cnt  verb_word noun_cnt  noun_word space  \
0               45.0        強い     70.0        られる    155.0         総理   0.0   
1               20.0       新しい     65.0        できる    105.0         国民   0.0   
2               15.0       大きい     55.0        いたす     80.0       野田総理   0.0   
3               10.0        重い     45.0        つくる     70.0       みずから   0.0   
4               10.0       力強い     35.0        続ける     60.0         責任   0.0   
5               10.0       美しい     35.0        果たす     60.0         政権   0.0   
6                5.0       著しい     35.0        述べる     50.0         約束   0.0   
7                5.0        軽い     30.0         示す     50.0       日米同盟   0.0   
8                5.0     恥ずかしい     30.0        迎える     45.0         外交   0.0   
9                5.0        若い     30.0         図る     40.0         結果   0.0   
10               5.0     揺るぎない     25.0         とる     35.0        皆さん   0.0   
11               5.0       つらい     25.0 

In [33]:
df3=(df2.iloc[:200,:]) 
print(df3)
df3.to_csv('./month_feature_word_200.csv',encoding='utf-8_sig')

     2012-10adj_cnt  adj_word verb_cnt  verb_word noun_cnt  noun_word space  \
0              45.0        強い     70.0        られる    155.0         総理   0.0   
1              20.0       新しい     65.0        できる    105.0         国民   0.0   
2              15.0       大きい     55.0        いたす     80.0       野田総理   0.0   
3              10.0        重い     45.0        つくる     70.0       みずから   0.0   
4              10.0       力強い     35.0        続ける     60.0         責任   0.0   
5              10.0       美しい     35.0        果たす     60.0         政権   0.0   
6               5.0       著しい     35.0        述べる     50.0         約束   0.0   
7               5.0        軽い     30.0         示す     50.0       日米同盟   0.0   
8               5.0     恥ずかしい     30.0        迎える     45.0         外交   0.0   
9               5.0        若い     30.0         図る     40.0         結果   0.0   
10              5.0     揺るぎない     25.0         とる     35.0        皆さん   0.0   
11              5.0       つらい     25.0       取り戻す   

In [32]:
df2.to_csv('./month_feature_word.csv',encoding='utf-8_sig')

In [56]:
#月日を抽出する実験コード

for file in glob.glob(r'C:\Users\icech\Desktop\share\Lab\2018_09_05\Docments\Abe_speech\Mecab\*'):
    print(file)
    month=re.sub( '.+Mecab.' ,'', file)
    month=re.sub('-01Prep.+', '' ,month)
    print(month)

C:\Users\icech\Desktop\share\Lab\2018_09_05\Docments\Abe_speech\Mecab\2012-10-01Prep.mecab
2012-10
C:\Users\icech\Desktop\share\Lab\2018_09_05\Docments\Abe_speech\Mecab\2012-11-01Prep.mecab
2012-11
C:\Users\icech\Desktop\share\Lab\2018_09_05\Docments\Abe_speech\Mecab\2013-01-01Prep.mecab
2013-01
C:\Users\icech\Desktop\share\Lab\2018_09_05\Docments\Abe_speech\Mecab\2013-02-01Prep.mecab
2013-02
C:\Users\icech\Desktop\share\Lab\2018_09_05\Docments\Abe_speech\Mecab\2013-03-01Prep.mecab
2013-03
C:\Users\icech\Desktop\share\Lab\2018_09_05\Docments\Abe_speech\Mecab\2013-04-01Prep.mecab
2013-04
C:\Users\icech\Desktop\share\Lab\2018_09_05\Docments\Abe_speech\Mecab\2013-05-01Prep.mecab
2013-05
C:\Users\icech\Desktop\share\Lab\2018_09_05\Docments\Abe_speech\Mecab\2013-06-01Prep.mecab
2013-06
C:\Users\icech\Desktop\share\Lab\2018_09_05\Docments\Abe_speech\Mecab\2013-10-01Prep.mecab
2013-10
C:\Users\icech\Desktop\share\Lab\2018_09_05\Docments\Abe_speech\Mecab\2013-11-01Prep.mecab
2013-11
C:\Users\i